# Avatar Redaction/Replacement - Identify and redact avatars and fill in their space with environment context

### Methodology
1. Avatar segmentation using a Pretrained FCN-ResNet model on DeepLabV3 architecture to remove avatar completely from footage.
2. Employ inpainting which uses a Two-stage adversarial model EdgeConnect to fill in the space which the avatars body has used.
3. Footage is converted into a sequence of frames and and the model is segmentation and inpainting models are applied to each frame.
4. Frames are stitched back together and saved to a local directory.
5. Depending if the user chooses to employ either CPU or GPU, the codebase will produce a lower-resolution video or a higher resolution video respectively
6. CPU will require long inference time, and is significantly slower than running on GPU. Model implementation is computationally demanding and overnight process of videos spanning a few minutes is requiered.

#### For installation instructions, please follow README after first initiating an environment via <em> anaconda </em>

##### The below code is a modification of https://github.com/sujaykhandekar/Automated-objects-removal-inpainter for the purposes of removing human avatars from a video feed (mp4)

## Package Import

In [1]:
import os
import random
import numpy as np
import torch
import argparse
from shutil import copyfile
from src.config import Config
from src.edge_connect import EdgeConnect
from argparse import Namespace
import sys
import shutil

import glob
import torchvision.transforms.functional as F
from torch.utils.data import DataLoader
from PIL import Image
import matplotlib.pyplot as plt
from imageio import imread
from skimage.feature import canny
from skimage.color import rgb2gray, gray2rgb
from src.utils import create_mask
from src.segmentor_fcn import segmentor,fill_gaps

import os
from torch.utils.data import DataLoader
from src.models import EdgeModel, InpaintingModel
from src.utils import Progbar, create_dir, stitch_images, imsave

import cv2
from cv2 import dnn_superres
import torch
import torchvision
import torchvision.transforms as T
import pathlib

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


## Main function

In [2]:
def main(modelType = None, res = None, folder = None):
    r"""starts the model

    """

    config = load_config(modelType, res, folder)
    
    # cuda visble devices
    os.environ['CUDA_VISIBLE_DEVICES'] = ','.join(str(e) for e in config.GPU)


    # init device
    if torch.cuda.is_available():
        config.DEVICE = torch.device("cuda")
        torch.backends.cudnn.benchmark = True   # cudnn auto-tuner
    else:
        config.DEVICE = torch.device("cpu")

    # set cv2 running threads to 1 (prevents deadlocks with pytorch dataloader)
    cv2.setNumThreads(0)

    # initialize random seed
    torch.manual_seed(config.SEED)
    torch.cuda.manual_seed_all(config.SEED)
    np.random.seed(config.SEED)
    random.seed(config.SEED)

    # build the model and initialize
    model = EdgeConnect(config)

    model.load()

    # model test
    print('begin redaction...\n')
    model.test(folder)

    ######################## remove input / output folders ###################################
    inputPath = 'examples/input/' + folder
    inputPath = os.path.join(os.getcwd(), inputPath)
    print("Removing " + inputPath + "...")
    shutil.rmtree(inputPath)
    
    
    outputPath = 'examples/output/' + folder
    outputPath = os.path.join(os.getcwd(), outputPath)
    print("Removing " + outputPath + "...")
    shutil.rmtree(outputPath)
    ######################## remove input / output folders ###################################

## Load Model configuration

In [3]:
def load_config(modelType=None, res = None, folder = None):
    r"""loads model config

    """

    # cpu mode
    if res == "cpu":
        config_dict = {
            "cpu": 'yes', 
            "edge": None, 
            "input": './examples/input/' + folder, 
            "model": modelType, 
            "output": './examples/output/' + folder, 
            "path": './checkpoints', 
            "remove": [15]
                   }
        args = Namespace(**config_dict)
        print(args)

    # gpu mode
    elif res == "gpu":
        config_dict = {
            "cpu": None, 
            "edge": None, 
            "input": './examples/input/' + folder, 
            "model": modelType, 
            "output": './examples/output/' + folder, 
            "path": './checkpoints', 
            "remove": [15],
                    }
        args = Namespace(**config_dict)
        
        print("Machine Configuration")
        print(args, "\n")
        
    else:
        print("please input either cpu or gpu as your mode")   

    ######################## create frames ########################
    vidcap = cv2.VideoCapture('./examples/source/' + folder + '.mp4')
    success,image = vidcap.read()
    count = 0
    inputPath = 'examples/input/' + folder
    inputPath = os.path.join(os.getcwd(), inputPath)
    try:  
        os.mkdir(inputPath)  
    except OSError as error:  
        print("path already exists") 
    while success:
      cv2.imwrite('./examples/input/' + folder + '/' + "frame%07d.jpg" % count, image)     # save frame as JPEG file      
      success,image = vidcap.read()
      count += 1
    ######################## create frames ########################
    
    #if path for checkpoint not given
    if args.path is None:
        args.path='./checkpoints'
    config_path = os.path.join(args.path, 'config.yml')
    
       # create checkpoints path if does't exist
    if not os.path.exists(args.path):
        os.makedirs(args.path)

    # copy config template if does't exist
    if not os.path.exists(config_path):
        copyfile('./config.yml.example', config_path)

    # load config file
    config = Config(config_path)
   
    # eval mode
    config.MODE = 3 # 1 train, 2 test, 3 eval
    config.MODEL = args.model if args.model is not None else 3
    config.OBJECTS = args.remove if args.remove is not None else [15]
    config.SEG_DEVICE = 'cpu' if args.cpu is not None else 'cuda'
    config.INPUT_SIZE = 256

    # inpout PATH
    if args.input is not None:
        config.TEST_FLIST = args.input
    
    if args.edge is not None:
        config.TEST_EDGE_FLIST = args.edge

    # output PATH
    if args.output is not None:
        config.RESULTS = args.output
    else: 
        if not os.path.exists('./results_images'):
            os.makedirs('./results_images')
        config.RESULTS = './results_images'

    return config

## Split Video into a sequence of frames

In [4]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, config, flist, edge_flist, augment=True, training=True):
        super(Dataset, self).__init__()
        self.augment = augment
        self.training = training
        self.data = self.load_flist(flist)
        self.edge_data = self.load_flist(edge_flist)

        self.input_size = config.INPUT_SIZE
        self.sigma = config.SIGMA
        self.edge = config.EDGE
        self.mask = config.MASK
        self.nms = config.NMS
        self.device = config.SEG_DEVICE
        self.objects = config.OBJECTS
        self.segment_net = config.SEG_NETWORK
        # in test mode, there's a one-to-one relationship between mask and image
        # masks are loaded non random
        

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        try:
            item = self.load_item(index)
        except:
            print('loading error: ' + self.data[index])
            item = self.load_item(0)

        return item

    def load_name(self, index):
        name = self.data[index]
        return os.path.basename(name)
        
    def load_size(self, index):
        img = Image.open(self.data[index])
        width,height=img.size
        return width,height


    def load_item(self, index):

        size = self.input_size

        # load image
        img = Image.open(self.data[index])
        
        
        # gray to rgb
        if img.mode !='RGB':
            img = gray2rgb(np.array(img))
            img=Image.fromarray(img)

        # resize/crop if needed
        img,mask=segmentor(self.segment_net,img,self.device,self.objects)
        img = Image.fromarray(img)

        # # print("show image from dataset before resize")
        # plt.imshow(img); plt.show()


        ######################### Determine aspect ratio ######################  
        # find aspect ratio
        width_og, height_og = img.size
        ratio = (width_og/height_og)

        if (ratio < 1):    
            size_W = int(size)
            size_H = int(452)      
        elif (ratio > 1):
            size_W = int(452)
            size_H = int(size)
        else:
            size_W = (size)
            size_H = (size)
        ######################### Determine aspect ratio ######################  
        
        # resize to square image
        img = np.array(img.resize((size_W, size_H), Image.LANCZOS))

        # print("show image from dataset after resize")
        # plt.imshow(img); plt.show()

        # create grayscale image
        img_gray = rgb2gray(np.array(img))

        # load mask
        mask = Image.fromarray(mask)

        # resize to square image
        mask = np.array(mask.resize((size_W, size_H), Image.LANCZOS))

        # # print("show mask from dataset")
        # plt.imshow(mask); plt.show()

        idx=(mask>0)
        mask[idx]=255
        #kernel = np.ones((5, 5), np.uint8)
        #opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        #closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
        mask=np.apply_along_axis(fill_gaps, 1, mask) #horizontal padding
        mask=np.apply_along_axis(fill_gaps, 0, mask) #vertical padding
        
        # load edge
        edge = self.load_edge(img_gray, index, mask)

        # augment data
        if self.augment and np.random.binomial(1, 0.5) > 0:
            img = img[:, ::-1, ...]
            img_gray = img_gray[:, ::-1, ...]
            edge = edge[:, ::-1, ...]
            mask = mask[:, ::-1, ...]

        return self.to_tensor(img), self.to_tensor(img_gray), self.to_tensor(edge), self.to_tensor(mask)

    def load_edge(self, img, index, mask):
        sigma = self.sigma

        # in test mode images are masked (with masked regions),
        # using 'mask' parameter prevents canny to detect edges for the masked regions
        mask = None if self.training else (1 - mask / 255).astype(np.bool)
        
        # canny
        if self.edge == 1:
            # no edge
            if sigma == -1:
                return np.zeros(img.shape).astype(np.float)

            # random sigma
            if sigma == 0:
                sigma = random.randint(1, 4)

            return canny(img, sigma=sigma, mask=mask).astype(np.float)

        # external
        else:
            imgh, imgw = img.shape[0:2]
            edge = imread(self.edge_data[index])
            edge = self.resized(edge, imgh, imgw)

            # non-max suppression
            if self.nms == 1:
                edge = edge * canny(img, sigma=sigma, mask=mask)

            return edge

    
    def to_tensor(self, img):
        img = Image.fromarray(img)
        img_t = F.to_tensor(img).float()
        return img_t


    def load_flist(self, flist):
        if isinstance(flist, list):
            return flist

        # flist: image file path, image directory path, text file flist path
        if isinstance(flist, str):
            if os.path.isdir(flist):
                flist = list(glob.glob(flist + '/*.jpg')) + list(glob.glob(flist + '/*.png'))
                flist.sort()
                return flist

            if os.path.isfile(flist):
                try:
                    return np.genfromtxt(flist, dtype=np.str, encoding='utf-8')
                except:
                    return [flist]

        return []

    def create_iterator(self, batch_size):
        while True:
            sample_loader = DataLoader(
                dataset=self,
                batch_size=batch_size,
                drop_last=True
            )

            for item in sample_loader:
                yield item


## Apply model to each frame 

#### If you wish to upscale with CPU, then chnage the conditional statment: <em> if (self.config.SEG_DEVICE != "gpu") </em> to anything which accepts CPU input. 

#### Addition note on upscaling 

EDSR [1]. This is the best performing model. However, it is also the biggest model and therefor has the biggest file size and slowest inference.

ESPCN [2]. This is a small model with fast and good inference. It can do real-time video upscaling (depending on image size).

FSRCNN [3]. This is also small model with fast and accurate inferenc

.
LapSRN [4]. This is a medium sized mo.e.

In [5]:
class EdgeConnect():
    def __init__(self, config):
        self.config = config

        if config.MODEL == 1:
            model_name = 'edge'
        elif config.MODEL == 2:
            model_name = 'inpaint'
        elif config.MODEL == 3:
            model_name = 'edge_inpaint'
        elif config.MODEL == 4:
            model_name = 'joint'

        self.debug = False
        self.model_name = model_name
        self.edge_model = EdgeModel(config).to(config.DEVICE)
        self.inpaint_model = InpaintingModel(config).to(config.DEVICE)

        # test mode
        self.test_dataset = Dataset(config, config.TEST_FLIST, config.TEST_EDGE_FLIST, augment=False, training=False)
      

        self.samples_path = os.path.join(config.PATH, 'samples')
        
        self.results_path = os.path.join(config.PATH, 'results')

        if config.RESULTS is not None:
            self.results_path = os.path.join(config.RESULTS)

        if config.DEBUG is not None and config.DEBUG != 0:
            self.debug = True

        self.log_file = os.path.join(config.PATH, 'log_' + model_name + '.dat')

    def load(self):
        if self.config.MODEL == 1:
            self.edge_model.load()

        elif self.config.MODEL == 2:
            self.inpaint_model.load()

        else:
            self.edge_model.load()
            self.inpaint_model.load()

    def save(self):
        if self.config.MODEL == 1:
            self.edge_model.save()

        elif self.config.MODEL == 2 or self.config.MODEL == 3:
            self.inpaint_model.save()

        else:
            self.edge_model.save()
            self.inpaint_model.save()


    def test(self, folder):
        self.edge_model.eval()
        self.inpaint_model.eval()

        model = self.config.MODEL
        create_dir(self.results_path)

        test_loader = DataLoader(
            dataset=self.test_dataset,
            batch_size=1,
        )


         ######################### upscale for GPU use ######################   
        if (self.config.SEG_DEVICE != "cpu"):

            # Create an SR object
            sr = dnn_superres.DnnSuperResImpl_create()
         ######################### upscale for GPU use ######################  
        
        index = 0
        for items in test_loader:        
            name = self.test_dataset.load_name(index)
            
            images, images_gray, edges, masks = self.cuda(*items)
            index += 1

            # edge model
            if model == 1:
                outputs = self.edge_model(images_gray, edges, masks)
                outputs_merged = (outputs * masks) + (edges * (1 - masks))

            # inpaint model
            elif model == 2:
                outputs = self.inpaint_model(images, edges, masks)
                outputs_merged = (outputs * masks) + (images * (1 - masks))

            # inpaint with edge model / joint model
            else:
                edges = self.edge_model(images_gray, edges, masks).detach()
                outputs = self.inpaint_model(images, edges, masks)
                outputs_merged = (outputs * masks) + (images * (1 - masks))

            output = self.postprocess(outputs_merged)[0]

            # print("load original image and get size")
            img = Image.open(self.test_dataset.__dict__['data'][0])
            # print(self.test_dataset.__dict__['data'][0])
            width_og, height_og = img.size

            path = os.path.join(self.results_path, name)
            print(index, name)
            imsave(output, path)
            
            ######################### upscale for GPU use ######################  
            if (self.config.SEG_DEVICE != "cpu"):
                # Read image
                image_sr = cv2.imread(path)
                # Read the desired model
                # model_sr = "EDSR_x3.pb"
                model_sr = "ESPCN_x4.pb"
                sr.readModel(model_sr)
                # Set the desired model and scale to get correct pre- and post-processing
                sr.setModel("espcn", 4)
                # Upscale the image
                result_sr = sr.upsample(image_sr)
                # Save the image
                cv2.imwrite(path, result_sr)
            ######################### upscale for GPU use ######################  

            if self.debug:
                edges = self.postprocess(1 - edges)[0]
                masked = self.postprocess(images * (1 - masks) + masks)[0]
                fname, fext = name.split('.')

                imsave(edges, os.path.join(self.results_path, fname + '_edge.' + fext))
                imsave(masked, os.path.join(self.results_path, fname + '_masked.' + fext))

        ########################## create output video ##############################
        outDirectory = 'examples/output/'
        
        outPath = outDirectory + folder + '/*.jpg'
        outPath = os.path.join(os.getcwd(), outPath)
        
        img = Image.open('./' + outDirectory + folder + '/' + name)
        
        width_og, height_og = img.size
        
        img_array = []
        
        for filename in glob.glob(outPath):
            img = cv2.imread(filename)
            height, width, layers = img.shape
            size = (width,height)
            img_array.append(img)
        out = cv2.VideoWriter(outDirectory + ('output-' + folder + '.mp4'),cv2.VideoWriter_fourcc(*'MP4V'), 30, size)
        for i in range(len(img_array)):
            out.write(img_array[i])
        out.release()
        ########################## create output video ##############################
        
        print('\nEnd redaction....')
        return output


    def log(self, logs):
        with open(self.log_file, 'a') as f:
            f.write('%s\n' % ' '.join([str(item[1]) for item in logs]))

    def cuda(self, *args):
        return (item.to(self.config.DEVICE) for item in args)

    def postprocess(self, img):
        # [0, 1] => [0, 255]
        img = img * 255.0
        img = img.permute(0, 2, 3, 1)
        return img.int()

Model is ran using the main function which can be configuered to run on CPU or GPU

# Parameters: main(x,y,z)

#### <strong>x</strong> : The modol will be applied to the video - <strong> 1 </strong> : <em> edge model </em> ,  <strong> 2 </strong> : <em> inpaint model </em>,  <strong> 3 </strong> : <em> edge-inpaint model </em>,  <strong> 4 </strong> : <em> joint model </em>

#### <strong>y</strong> : Type of machine which the redation is being performed - <strong> cpu </strong> : <em> CPU mode </em> (low freame-rate, low-resolution video), <strong> gpu </strong> : <em> GPU mode </em> (full frame-rate, full resolution via upscaling)

#### <strong>z</strong> : Name of source video - Source video will need to be stored in the <strong> \examples\source\ </strong> folder within the repositry. 

Once the model has completed redaction / inpainting, then the output video will be stored in <strong> \examples\output\ </strong>, sharing the same name as the source video

In [6]:
# the following function passes three arguments to the main function
# 3 : an edge-inpaint model will be applied to the video
# cpu : a cpu friendly version of the codebase will be ran on the video input - this will result in a slower inference time and lower-resolution output video
# shopping2 : shopping2 is the name of the input file located in the project directory \examples\source\shopping2.mp4

main(3, "cpu", "shopping1")

Namespace(cpu='yes', edge=None, input='./examples/input/shopping1', model=3, output='./examples/output/shopping1', path='./checkpoints', remove=[15])
Loading EdgeModel generator...
Loading InpaintingModel generator...
begin redaction...



C:\Users\gomolina\Anaconda3\envs\objRemover\lib\site-packages\torchvision\transforms\functional.py:74: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  img = torch.ByteTensor(torch.ByteStorage.from_buffer(pic.tobytes()))
C:\Users\gomolina\Anaconda3\envs\objRemover\lib\site-packages\torchvision\transforms\functional.py:70: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\to

1 frame0000000.jpg
2 frame0000001.jpg
3 frame0000002.jpg
4 frame0000003.jpg
5 frame0000004.jpg
6 frame0000005.jpg
7 frame0000006.jpg

End redaction....
Removing C:\Users\gomolina\OneDrive - Deloitte (O365D)\Documents\GitHub\Automated-objects-removal-inpainter\examples/input/shopping1...
Removing C:\Users\gomolina\OneDrive - Deloitte (O365D)\Documents\GitHub\Automated-objects-removal-inpainter\examples/output/shopping1...


#### The below footage was passed through the model in GPU mode. I.e: main(3, "cpu", "shopping2")

<video width="426" src="examples\source\shopping2.mp4" controls title="Title"></video>

#### The below footage is the result

<video width="426" src="examples\output\output-shopping2.mp4 " controls title="Title">

### Here are both video put together to run simultaneously

<video width="426" src="examples\demo\shopping2.mp4" controls title="Title"></video>

### Future Work

#### 1 : Source pre-trained models to include indoor door settings. 


The pretrained model performs at it’s weakest when encountering indoor / irregular camera angled settings. Sourcing pre-trained models or training a model with labelled cctv footage is ideal so it best handles those environments when encountered. 

#### 2 : Ability to redact specific avatars 


Consenting parties may want to appear in the video footage once the film had pasted through the model. 

Solution can be configured to only redact figures who are not wearing high-vis jackets (employee of a work site) and blur everyone else (general public). 

#### 3 : Ability to redact specific avatars 

Feed results of the previous frame into the next frame to smooth out video 

The video output can sometimes look jittery due to applying the inpainting model to each frame individually. 

A way to enhance the output video would be to use the result of an impainted frame (the model has redacted the avatar and filled in the gap with environment context) and use that filled in frame as context for the next framvideo 